In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import json
import requests
import time
import datetime
import os
import binance
import matplotlib.pyplot as plt

def get_kline(startTime,limit,symbol,interval):
    if interval[-1] == 'm':
        endTime = int(startTime+(limit*int(interval[:-1])*60*1500))
        
    data = requests.get('https://fapi.binance.com/fapi/v1/klines', params={"symbol" : symbol,
                                                        "interval" : interval, 
                                                        "startTime" : startTime,
                                                        "endTime"   : endTime,
                                                        "limit": limit}).json()
    #data  = pd.DataFrame(data)
    data  = pd.DataFrame(data,columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', 'no_of_trades', 'tb_base_vol', 'tb_quote_vol', 'ignore' ])
    data.iloc[:,1:]=data.iloc[:,1:].astype(float)
    data['time'] = pd.to_datetime(data['timestamp'], unit='ms')
    return data



def get_kline_lookback(symbol,interval,limit,lookback_date):
    date = datetime.datetime.strptime(lookback_date, "%d/%m/%Y")
    timestamp_lookback = datetime.datetime.timestamp(date)

    time_del = timestamp_now-timestamp_lookback
    timestep = int(timeframe[:-1])
    no_loop  = round(time_del/(timestep*60*1000))+1 

    #download the historical price, you can change the candle resolution
    kline = get_kline(startTime=timestamp_lookback*1000,limit=1000,symbol=ticker,interval=timeframe)
    for i in range(no_loop):
        kline_temp = get_kline(startTime=kline['timestamp'].values[-1],limit=1000,symbol=ticker,interval=timeframe)
        kline=pd.concat([kline,kline_temp]).drop_duplicates().reset_index(drop=True)
        
    kline=pd.concat([kline,kline_temp]).drop_duplicates().reset_index(drop=True)
    kline = kline.groupby('timestamp').mean().reset_index() #getting rid of the repeated values
    kline['time'] = pd.to_datetime(kline['timestamp'], unit='ms')
    return kline


In [2]:
timestamp_now = datetime.datetime.now().timestamp()
ticker = 'BTCUSDT'
timeframe = '5m'
lookback_date = "15/05/2022"
#download kline
kline = get_kline_lookback(symbol=ticker,interval=timeframe,limit=1000,lookback_date=lookback_date)

In [7]:


#plot the regression line the below subplot
#subplot with 2 rows and 1 column using plotly

fig = make_subplots(rows=3, cols=1, row_heights=[0.5, 0.15,0.15], vertical_spacing=0.02,shared_xaxes=True)

#add title of the plot
fig.update_layout(title_text=f'{ticker} Risk Analysis')
fig.add_trace(go.Scatter(x=kline['time'], y=kline['close'],name=' Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=kline['time'], y=close_price_regression['close_regression'], name='Regression Gradient'), row=2, col=1)



#fig.add_trace(go.Scatter(x=kline['time'], y=percentage_change_std, name='mean'), row=3, col=1)
#add EMA of percentage change std
fig.add_trace(go.Scatter(x=kline['time'], y=percentage_change_std.ewm(span=50).mean(), name='Volatility'), row=3, col=1)

#custom plot width and height
fig.update_layout(width=1500, height=800)
#share x-axis

fig.show()